# Loading Fake Timeseries Surface Data

This notebook is designed to explore some functionality with loading DataFiles and using Loaders.

This example will require some extra optional libraries, including nibabel and nilearn! Note: while nilearn is not imported, when trying to import SingleConnectivityMeasure, if nilearn is not installed, this will give an ImportError.

We will also use fake data for this example - so no special datasets required!

In [1]:
import BPt as bp
import nibabel as nib
import numpy as np
import pandas as pd
import os

In [2]:
def save_fake_timeseries_data():
    '''Save fake timeseries and fake surface data.'''
    
    X = np.random.random(size = (20, 100, 10242))
    os.makedirs('fake_time_data', exist_ok=True)

    for x in range(len(X)):
        np.save('fake_time_data/' + str(x) + '_lh', X[x])
    for x in range(len(X)):
        np.save('fake_time_data/' + str(x) + '_rh', X[x])
        
save_fake_timeseries_data()

In [3]:
# Init a Dataset
data = bp.Dataset()

Next, we are interested in loading in the files to the dataset as data files. There are a few different ways to do this, but we will use the method add_data_files. We will try and load the timeseries data first.

First we need a dictionary mapping desired column name to location or a file glob (which is easier so let's use that).

In [4]:
# The *'s just mean wildcard
files = {'timeseries_lh': 'fake_time_data/*_lh*',
         'timeseries_rh': 'fake_time_data/*_rh*'}

# Now let's try loading with 'auto' as the file to subject function
data.add_data_files(files, 'auto')

,timeseries_lh,timeseries_rh
13_lh,Loc(0),nan
9_lh,Loc(1),nan
8_lh,Loc(2),nan
2_lh,Loc(3),nan
16_lh,Loc(4),nan
11_lh,Loc(5),nan
6_lh,Loc(6),nan
7_lh,Loc(7),nan
1_lh,Loc(8),nan
17_lh,Loc(9),nan


We can see 'auto' doesn't work for us, so we can try writing our own function instead.

In [5]:
def file_to_subj(loc):
    return loc.split('/')[-1].split('_')[0]

# Actually load it this time
data = data.add_data_files(files, file_to_subj)
data

,timeseries_lh,timeseries_rh
13,Loc(0),Loc(36)
9,Loc(1),Loc(27)
8,Loc(2),Loc(37)
2,Loc(3),Loc(35)
16,Loc(4),Loc(38)
11,Loc(5),Loc(20)
6,Loc(6),Loc(29)
7,Loc(7),Loc(31)
1,Loc(8),Loc(26)
17,Loc(9),Loc(39)


What's this though? Why are the files showing up as Loc(int). Whats going on is that the data files are really stored as just integers, see:

In [6]:
data['timeseries_lh']

13     0.0
9      1.0
8      2.0
2      3.0
16     4.0
11     5.0
6      6.0
7      7.0
1      8.0
17     9.0
19    10.0
15    11.0
10    12.0
3     13.0
14    14.0
0     15.0
18    16.0
5     17.0
4     18.0
12    19.0
Name: timeseries_lh, dtype: float64

They correspond to locations in a stored file mapping (note: you don't need to worry about any of this most of the time)

In [7]:
data.file_mapping[0], data.file_mapping[1], data.file_mapping[2]  

(DataFile(loc='/home/sage/BPt/Examples/Short_Examples/fake_time_data/13_lh.npy'),
 DataFile(loc='/home/sage/BPt/Examples/Short_Examples/fake_time_data/9_lh.npy'),
 DataFile(loc='/home/sage/BPt/Examples/Short_Examples/fake_time_data/8_lh.npy'))

Let's add a fake target to our dataset now

In [8]:
data['t'] = np.random.random(len(data))
data.set_target('t', inplace=True)
data

,timeseries_lh,timeseries_rh
13,Loc(0),Loc(36)
9,Loc(1),Loc(27)
8,Loc(2),Loc(37)
2,Loc(3),Loc(35)
16,Loc(4),Loc(38)
11,Loc(5),Loc(20)
6,Loc(6),Loc(29)
7,Loc(7),Loc(31)
1,Loc(8),Loc(26)
17,Loc(9),Loc(39)


Next we will generate a Loader to apply a parcellation, then extract a measure of connectivity.

In [9]:
from BPt.extensions import SurfLabels

lh_parc = SurfLabels(labels='data/lh.aparc.annot', vectorize=False)
rh_parc = SurfLabels(labels='data/rh.aparc.annot', vectorize=False)

We can see how this object works on example data first.

In [10]:
ex_lh = data.file_mapping[0].load()
ex_lh.shape

(100, 10242)

In [11]:
trans = lh_parc.fit_transform(ex_lh)
trans.shape

(100, 35)

We essentially get a reduction from 10242 features to 35.

Next, we want to transform the matrix into a correlation matrix.

In [12]:
from BPt.extensions import SingleConnectivityMeasure
scm = SingleConnectivityMeasure(kind='covariance', discard_diagonal=True, vectorize=True)

In [13]:
scm.fit_transform(trans).shape

(595,)

The single connectivity measure is just a wrapper designed to let the ConnectivityMeasure from nilearn work with a single subject's data at a time.

Next, let's use the input special Pipe wrapper to compose these two objects into their own pipeline

In [14]:
lh_loader = bp.Loader(bp.Pipe([lh_parc, scm]), scope='_lh')
rh_loader = bp.Loader(bp.Pipe([rh_parc, scm]), scope='_rh')

Define a simple pipeline with just our loader steps, then evaluate with mostly default settings.

In [15]:
pipeline = bp.Pipeline([lh_loader, rh_loader, bp.Model('linear')])

results = bp.evaluate(pipeline, data)
results

Folds:   0%|          | 0/5 [00:00<?, ?it/s]

BPtEvaluator
------------
mean_scores = {'explained_variance': -0.11474424067480002, 'neg_mean_squared_error': -0.0806213742654183}
std_scores = {'explained_variance': 0.4045165213520749, 'neg_mean_squared_error': 0.03933087964492685}

Saved Attributes: ['estimators', 'preds', 'timing', 'train_indices', 'val_indices', 'feat_names', 'ps', 'mean_scores', 'std_scores', 'weighted_mean_scores', 'scores', 'fis_', 'coef_']

Avaliable Methods: ['get_preds_dfs', 'get_fis', 'get_coef_', 'permutation_importance']

Evaluated with:
ProblemSpec(problem_type='regression',
            scorer={'explained_variance': make_scorer(explained_variance_score),
                    'neg_mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False)},
            subjects='all', target='t')

Don't be discouraged that this didn't work, we are after all trying to predict random noise with random noise ... 

In [16]:
# These are the steps of the pipeline
fold0_pipeline = results.estimators[0]
for step in fold0_pipeline.steps:
    print(step[0])

loader_pipe0
loader_pipe1
linear regressor


We can investigate pieces, or use special functions like

In [17]:
results.get_X_transform_df(data, fold=0)

,timeseries_rh_0,timeseries_rh_1,timeseries_rh_2,timeseries_rh_3,timeseries_rh_4,timeseries_rh_5,timeseries_rh_6,timeseries_rh_7,timeseries_rh_8,timeseries_rh_9,...,timeseries_lh_585,timeseries_lh_586,timeseries_lh_587,timeseries_lh_588,timeseries_lh_589,timeseries_lh_590,timeseries_lh_591,timeseries_lh_592,timeseries_lh_593,timeseries_lh_594
0,3.974341e-05,0.000005,0.000024,-3.888821e-07,-6.604009e-05,-4.549132e-06,-0.000067,0.000038,0.000010,0.000009,...,3.356934e-06,-0.000027,-0.000014,1.275423e-07,3.361936e-06,1.226298e-05,5.861499e-06,0.000071,0.000015,0.000006
1,-7.350213e-05,-0.000036,-0.000047,-6.572900e-05,6.707736e-06,3.090695e-05,-0.000019,0.000014,-0.000001,-0.000033,...,-1.260432e-05,-0.000029,0.000022,6.801169e-06,-1.132151e-05,2.388484e-06,-2.882874e-05,0.000099,0.000027,0.000016
2,-6.199951e-05,0.000037,-0.000052,-5.350310e-05,-6.938637e-06,4.931621e-05,0.000105,-0.000066,0.000072,-0.000009,...,-1.729334e-05,-0.000057,-0.000016,6.953608e-06,-3.087901e-05,9.642614e-06,8.896876e-06,0.000032,-0.000007,0.000020
3,2.888482e-06,0.000024,-0.000036,-1.941099e-05,-2.149982e-05,4.349853e-05,-0.000016,0.000015,-0.000004,-0.000016,...,-5.483235e-07,0.000053,0.000010,1.437846e-05,-1.950094e-05,-1.562847e-05,-2.633812e-05,-0.000006,0.000032,-0.000025
4,-4.910675e-05,-0.000024,-0.000035,-2.797020e-05,-1.055248e-04,1.539235e-05,-0.000031,-0.000113,0.000001,0.000007,...,-2.492312e-05,-0.000003,-0.000010,2.617981e-05,-8.456509e-06,-2.359148e-05,-1.261547e-06,-0.000014,0.000085,-0.000040
5,-9.493835e-05,0.000017,-0.000093,-2.065541e-05,-1.053112e-04,-4.606653e-05,0.000032,0.000148,-0.000077,0.000007,...,1.225922e-05,-0.000009,-0.000011,1.866004e-05,-1.499551e-06,-7.293683e-06,-4.005567e-05,-0.000092,0.000007,-0.000110
6,1.284061e-04,-0.000003,0.000004,2.104696e-05,-3.381504e-05,1.658760e-05,0.000045,-0.000037,-0.000052,0.000006,...,6.242599e-06,-0.000019,-0.000039,1.076647e-06,8.934646e-06,1.582536e-05,1.727403e-05,0.000020,-0.000002,-0.000032
7,1.209134e-04,-0.000034,-0.000082,6.786592e-06,-9.119613e-05,-2.468941e-05,0.000006,0.000053,-0.000006,-0.000072,...,1.274468e-05,-0.000030,-0.000019,-9.027374e-07,1.726328e-07,8.771389e-06,2.121292e-06,0.000062,-0.000095,-0.000016
8,4.086714e-05,-0.000013,-0.000032,6.498699e-05,2.951521e-05,-1.142601e-05,0.000069,-0.000026,0.000007,0.000050,...,-9.006227e-07,-0.000009,-0.000048,6.410512e-06,6.262172e-06,-5.239753e-06,-1.596085e-05,-0.000073,0.000016,0.000020
9,-8.063259e-06,0.000035,-0.000026,-1.303488e-05,-3.501850e-05,2.485596e-05,0.000054,0.000015,-0.000024,-0.000026,...,-2.844877e-06,0.000017,-0.000037,1.396516e-05,1.555043e-05,4.406335e-06,-2.813589e-05,0.000020,-0.000096,-0.000033
